In [6]:
#pip install pingouin

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
import pingouin as pg
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
import os

In [2]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)

In [3]:
os.chdir('C:/Users/Ryo/OneDrive/Desktop/Master Thesis/master_thesis/study3')

# data preparation

## prep for the response df

In [4]:
file_loc = 'raw/Study3 - final_July_18_2024_02_35_final.xlsx'
df = pd.read_excel(file_loc)
df

,ResponseId,Ads_OP_1_Reversed,Ads_OP_2_Reversed,Ads_OP_3_Reversed,Ads_OP_4_Reversed,Ads_OP_5_Reversed,Ads_OP_6_Reversed,Ads_CO_1_Normal,Ads_CO_2_Normal,Ads_CO_3_Normal,Ads_CO_4_Normal,Ads_CO_5_Normal,Ads_CO_6_Normal,Ads_EX_1_Reversed,Ads_EX_2_Reversed,Ads_EX_3_Reversed,Ads_EX_4_Reversed,Ads_EX_5_Reversed,Ads_EX_6_Reversed,Ads_AG_1_Normal,Ads_AG_2_Normal,Ads_AG_3_Normal,Ads_AG_4_Normal,Ads_AG_5_Normal,Ads_AG_6_Normal,Desc_OP_1_Normal,Desc_CO_1_Reversed,Desc_EX_1_Normal,Desc_AG_1_Reversed
0,5db4f0b63e33f2000dd54016,22,19,14,16,17,15,21,22,18,18,18,15,24,25,23,22,21,17,70,69,69,68,65,61,75,26,75,22
1,66294a585f1cf3fdaeb80120,28,50,47,72,50,41,34,50,50,59,46,21,46,50,58,62,55,38,56,50,55,64,58,60,91,90,80,40
2,665a52573b9527ce0011158b,10,10,10,10,10,10,0,0,0,0,0,0,10,10,10,10,10,10,61,60,61,61,61,62,100,100,100,42
3,651ebcca48c60acc82f1f2bd,60,50,63,90,90,89,31,50,34,50,39,37,62,50,65,80,70,79,62,50,54,67,67,77,82,82,82,34
4,6634fb892af2e227ab65f004,80,50,90,85,90,85,80,50,75,50,75,87,95,50,95,50,95,95,70,60,70,67,70,70,10,80,60,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,64d52f62a1f1a7371760fe4f,82,64,86,72,79,90,40,44,24,37,31,15,39,31,21,21,13,13,81,73,91,68,81,89,20,70,83,32
88,66622736b3fcb6b8f3c04fdd,88,86,92,83,86,85,28,41,11,28,28,16,81,78,87,81,75,86,68,66,70,61,70,66,76,26,68,30
89,6660680acd027329cecdf2b8,37,33,35,49,47,46,61,56,67,53,74,86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
90,5d215a1bbf7f840019701939,53,52,70,62,75,82,35,49,28,40,31,33,69,69,77,64,69,68,53,54,55,51,50,50,51,52,53,53


In [18]:
print(f"shape {df.shape}")

shape (90, 29)


In [15]:
file_loc = 'raw/20240721_prolific_export.csv'
target_respondants = pd.read_csv(file_loc)
target_respondants = target_respondants[target_respondants['Status'] == 'APPROVED']['Participant id'].unique().tolist()
len(target_respondants)

90

In [17]:
study3_respondants = df['ResponseId'].tolist()
print(f"original respondse counts: {len(study3_respondants)}")

# filter df to target participants
# Count non-null values for each row
df['non_null_count'] = df.notna().sum(axis=1)

# Sort by 'ResponseId' and 'non_null_count' (descending), then drop duplicates
df = df.sort_values(['ResponseId', 'non_null_count'], ascending=[True, False]).drop_duplicates('ResponseId')

# Remove the temporary 'non_null_count' column
df = df.drop('non_null_count', axis=1)

# Filter to keep only target respondents
df = df[df['ResponseId'].isin(target_respondants)]
print(f"final respondse counts: {len(df['ResponseId'])}")


original respondse counts: 91
final respondse counts: 90


## prep for socioecnomic and big5 scores

In [23]:
big5_file_loc = 'raw/big5_scores.csv'
big5_df = pd.read_csv(big5_file_loc)
big5_df.tail(5)

,ResponseId,Extraversion,Agreeableness,Conscientiousness,Neuroticism,Open_Mindedness
99,60a062ed4c4334abbbf32323,3.333333,3.000000,3.166667,3.666667,4.166667
100,5f3ac1732efa0a74f975b1a8,3.500000,4.833333,4.333333,1.000000,3.000000
101,6601b51675d287d6a62f11c4,3.666667,4.833333,4.333333,1.166667,4.666667
102,656f2e99bd2939f9b3f9d090,3.833333,4.000000,4.666667,1.666667,4.000000
103,6658822c0f5b1367a1585ee1,4.500000,4.666667,5.000000,1.333333,3.166667


In [24]:
socioecono_file_loc = 'raw/socioecono_scores.csv'
socioecono_df = pd.read_csv(socioecono_file_loc)
socioecono_df.tail(5)

,ResponseId,gender,age,education,race,employment,income
99,60a062ed4c4334abbbf32323,2,27,4,2,1,1
100,5f3ac1732efa0a74f975b1a8,2,30,2,1,6,5
101,6601b51675d287d6a62f11c4,2,55,5,2,99,11
102,656f2e99bd2939f9b3f9d090,1,43,5,2,7,12
103,6658822c0f5b1367a1585ee1,2,43,5,2,7,8


In [25]:
# filter socioecono_df and big5_df to study3 participants
socioecono_df = socioecono_df[socioecono_df['ResponseId'].isin(target_respondants)]
big5_df = big5_df[big5_df['ResponseId'].isin(target_respondants)]
print(f"row # for socioecono_df : {len(socioecono_df['ResponseId'])}, row # for big5_df : {len(big5_df['ResponseId'])}")

row # for socioecono_df : 90, row # for big5_df : 90


# general analysis

In [26]:
print(f"mean age {socioecono_df['age'].mean()} with sd {socioecono_df['age'].std()}, female % {socioecono_df[socioecono_df['gender']==1]['gender'].sum()/socioecono_df.shape[0]}")

mean age 40.86666666666667 with sd 11.74466170869495, female % 0.4777777777777778
